In [1]:
import time

## Data wrangling
import pandas as pd
import numpy as np

## Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import warnings
warnings.filterwarnings('ignore')

## Machine learning
from sklearn.preprocessing import normalize, Normalizer
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, f1_score, recall_score, roc_auc_score, roc_curve, auc

## Spatial data wrangling
import geopandas as gpd
from shapely.geometry import Point, Polygon

import multiprocessing
print ('Number of CPUS: ' +str(multiprocessing.cpu_count()))

import datetime as dt     
date = dt.datetime.today().strftime("%Y/%m/%d")
print (date)

Number of CPUS: 2
2018/07/11


## Setting Parameters

In [2]:
outcomes = ['activity']

models_to_run = ['RF']

date_offset = [3]

target_date = '2017/09/01' #pd.to_datetime('today').strftime("%Y/%m/%d")
print(target_date)
target_date = pd.to_datetime(target_date)

2017/09/01


In [3]:
clfs = {
    'RF': RandomForestClassifier(n_estimators=50, n_jobs=-1),
    'LR': LogisticRegression(penalty='l1', C=1e5),
    'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)
        }

In [4]:
grid = {
    'RF':{'n_estimators': [125,250], 'max_depth': [3,5,10],
          'max_features': [0.25,.5],'min_samples_split': [2],'criterion':['gini']},  
    'LR': { 'penalty': ['l1','l2'],'C': [0.01,0.1,1,10]}, 
    'GB': {'n_estimators': [250], 'learning_rate' : [0.1],
           'subsample' : [0.5], 'max_depth': [10]} 
       }

## Defining Functions

In [5]:
def data_prep(date_offset=1):
    
    blks = pd.read_csv('data/census_blocks.csv.gz')
    blks['GEOID'] = blks['GEOID'].astype(str)
    blks = blks.set_index('GEOID')

    geo_rats = pd.read_csv('data/rats_to_blocks.csv.gz')
    geo_rats['GEOID'] = geo_rats['GEOID'].astype(str)

    geo_bbl = pd.read_csv('data/bbls_to_blocks.csv.gz')
    geo_bbl['GEOID'] = geo_bbl['GEOID'].astype(str)

    geo_permits = pd.read_csv('data/permits_to_blocks.csv.gz')
    geo_permits['GEOID'] = geo_permits['GEOID'].astype(str)

    geo_mar = pd.read_csv('data/address_units_to_blocks.csv.gz')
    geo_mar['GEOID'] = geo_mar['GEOID'].astype(str)
    geo_mar = geo_mar.set_index('GEOID')

    geo_cama = pd.read_csv('data/cama_to_blocks.csv.gz')
    geo_cama['GEOID'] = geo_cama['GEOID'].astype(str)
    geo_cama = geo_cama.set_index('GEOID')

    dcgis = pd.read_csv('data/env_features.csv.gz')
    dcgis['GEOID'] = dcgis['GEOID'].astype(str)
    dcgis = dcgis.set_index('GEOID')

    years = np.sort(geo_rats.year.unique())
    years = years[~np.isnan(years)]
    
    data = pd.DataFrame()
    for year in years:

        year = int(year)
        this_year = geo_rats[geo_rats.year==year]
        months = np.sort(this_year.month.astype(int).unique())

        for month in months:
            month = int(month)
            date = pd.datetime(year, month, 1)
                        
            ## Get outcome for month
            mrats = geo_rats[(pd.to_datetime(geo_rats.SERVICEORDERDATE) >= date) 
                             & (pd.to_datetime(geo_rats.SERVICEORDERDATE) < date + pd.DateOffset(months=date_offset))]
            mrats_3 = geo_rats[(pd.to_datetime(geo_rats.SERVICEORDERDATE) <= date) 
                             & (pd.to_datetime(geo_rats.SERVICEORDERDATE) > date - pd.DateOffset(months=3))]
            mrats = mrats.fillna(0)
            mrats_3 = mrats_3.fillna(0)
            rats_by_block = pd.DataFrame(mrats.groupby(['GEOID'])[['calls', 'activity']].sum())
            rats_by_block['calls'] = 1*(rats_by_block['calls'] > 0)
            rats_by_block['activity'] = 1*(rats_by_block['activity'] > 0)

            ## Rat call in last 3 months
            rats_past_3m = pd.DataFrame(mrats_3.groupby(['GEOID'])[['calls']].sum())
            rats_past_3m['call_past_3m'] = 1*(rats_past_3m['calls'] > 0)
            rats_past_3m = rats_past_3m.drop(['calls'], axis=1)
            
            ## Merge to full set Census blocks and fill NA's with zeroes 
            mdata = blks.merge(rats_by_block, how='left', left_index=True, right_index=True)
            mdata = mdata.merge(rats_past_3m, how='left', left_index=True, right_index=True)

            ## Get month and year vars
            mdata['year'] = year
            mdata['month']= month
            mmonths = pd.get_dummies(mdata['month'], prefix='month')
            myears = pd.get_dummies(mdata['year'], prefix='year')
            mdata = mdata.merge(mmonths, how='left', left_index=True, right_index=True)
            mdata = mdata.merge(myears, how='left', left_index=True, right_index=True)
            mdata['yearmonth'] = mdata['year'] + (mdata['month']*1.0/12)
            mdata['yearmonth'].describe().T
            
            ## Adding BBLs that started before the beginning of this month but ended after
            businesses = ['bbl_food',  'bbl_hotel', 'bbl_pool', 'bbl_restaurant', 'bbl_storage', 
                        'bbl_multifamily_rental', 'bbl_single_family_rental', 'bbl_two_family_rental', 'bbl_waste']
            mbbls = geo_bbl[(pd.to_datetime(geo_bbl.license_start_date) <= date) & (pd.to_datetime(geo_bbl.license_exp_date) > date)]
            bbls_by_block = pd.DataFrame(mbbls.groupby('GEOID')[businesses].sum())
            mdata = mdata.merge(bbls_by_block, how='left', left_index=True, right_index=True)

            ## Adding permits issued before the date but not more than 12 months before
            permits = ['dcrapermit_new_building', 'dcrapermit_demolition', 'dcrapermit_raze', 'dcrapermit_addition', 
                   'dcrapermit_retain_wall', 'dcrapermit_excavation', 'dcrapermit_pool', 'dcrapermit_garage']
            mpermits = geo_permits[(pd.to_datetime(geo_permits.issue_date) <= date) & ((pd.to_datetime(geo_permits.issue_date) + pd.DateOffset(months=12)) > date)]
            permits_by_block = pd.DataFrame(mpermits.groupby('GEOID')[permits].sum())
            mdata = mdata.merge(permits_by_block, how='left', left_index=True, right_index=True)

            ## Add DCGIS data (prepped in separate script)
            mdata = mdata.merge(geo_mar, how='left', left_index=True, right_index=True)

            ## Add DCGIS data (prepped in separate script)
            mdata = mdata.merge(geo_cama, how='left', left_index=True, right_index=True)

            ## Add DCGIS data (prepped in separate script)
            mdata = mdata.merge(dcgis, how='left', left_index=True, right_index=True)

            ## Append to training data
            data = data.append(mdata)
            print ('Finished prepping data for '+str(month)+'-'+str(year)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

    data = data.fillna(0)
    data.to_csv('data/rodent_model_data.csv', compression = 'gzip')
    return data

In [6]:
def pull_data(data, outcome, year, month, date_offset=1, verbose=False):
    
    year = int(year)
    month = int(month)
    date = pd.datetime(year, month, 1)
        
    years = np.sort(data.year.astype(int).unique())
        
    test = data[(data.year==year) & (data.month==month)]    
    
    ## Get training data
    train = pd.DataFrame()
    for y in years: 
        y = int(y)
        months = np.sort(data[data.year==y].month.astype(int).unique())

        for m in months: 
            
            d = pd.datetime(y, m, 1) ## Get the date
            
            ## If the date is less than or equal to the date minus the offset then add it to the training set
            if d <= (date - pd.DateOffset(months=date_offset)): 
                m = int(m)
                mtrain = data[(data.year==y) & (data.month==m)]
                train = train.append(mtrain)
                
            else: ## Otherwise if it's a later month, skip it
                continue
    
    test = test.drop(['year', 'month'], axis=1)
    train = train.drop(['year', 'month'], axis=1)
    test = test.fillna(0)
    train = train.fillna(0)
    #### Split out y data from x
    if outcome == 'calls':
        ## Rebalance sample for calls model
        pos = train[train.calls==1] ## All positive cases
        neg = train[train.calls==0].sample(n=len(pos), replace=True) ## Bootstrap sample of negative cases equivalent to size to positive cases
        train = pos.append(neg) ## Append negative cases to positive cases

        ## Data for Calls models
        X_train = train.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_train = train[['calls']]

        X_test = test.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_test = test[['calls', 'call_past_3m']]

    if outcome == 'reinspect':
        ## Rebalance sample for calls model
        train = train[train.call_past_3m==1]
        test = test[test.call_past_3m==1]
        pos = train[train.calls==1] ## All positive cases
        neg = train[train.calls==0].sample(n=len(pos), replace=True) ## Bootstrap sample of negative cases equivalent to size to positive cases
        train = pos.append(neg) ## Append negative cases to positive cases

        ## Data for Calls models
        X_train = train.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_train = train[['calls']]

        X_test = test.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_test = test[['calls', 'call_past_3m']]

        
    elif outcome == 'activity':
        ## Restrict sample to locations that were 
        train = train[train.calls==1]

        ## Data fro Activity models
        X_train = train.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_train = train[['activity']]

        X_test = test.drop(['calls', 'activity', 'call_past_3m'], axis=1)
        y_test = test[['calls', 'activity', 'call_past_3m']]
    
    if verbose==True:
        print ('Finished pulling data for '+str(month)+'-'+str(year)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))
    
    return X_train, y_train, X_test, y_test

In [7]:
def train_model(outcome, year, month, clf, model, params, X_train, y_train, X_test, y_test):
    
    results = pd.DataFrame()

    ## if Logistic Regression, normalize data
    if model=='LR':
        cont_feat = []
        for c in X_train.columns:
            if len(X_train[c].unique()) > 2:
                cont_feat.append(c)
            else: continue

        norm = Normalizer()
        fnorm = norm.fit(X_train[cont_feat])
        X_train[cont_feat] = fnorm.transform(X_train[cont_feat])
        X_test[cont_feat] = fnorm.transform(X_test[cont_feat])
        
    ## Fit classifier
    clf.fit(X_train, y_train.values.ravel())
    
    ## Get predictions and predicted probabilities
    pred = pd.DataFrame(clf.predict(X_test), index=X_test.index)
    pred.columns = ['predicted']
    predp = pd.DataFrame(clf.predict_proba(X_test)[:,1], index=X_test.index)
    predp.columns = ['predicted_prob']

    ## Merge together to get test stats
    predicted = y_test.merge(pred, how='left', left_index=True, right_index=True)
    predicted = predicted.merge(predp, how='left', left_index=True, right_index=True)
    predicted = predicted.sort_values(['predicted_prob'], ascending=False)
    
    if outcome=='calls':
        ptop = predicted.head(n=100)
        precision_at_n = precision_score(ptop[outcome], ptop['predicted'])
        recall_at_n = recall_score(ptop[outcome], ptop['predicted'])
        roc_auc = roc_auc_score(predicted[outcome], predicted['predicted_prob'])
        
        prob_avg = predicted.predicted_prob.mean()
        prob_med = predicted.predicted_prob.quantile(.5)
        prob_min = predicted.predicted_prob.min()
        prob_max = predicted.predicted_prob.max()
        prob_q25 = predicted.predicted_prob.quantile(.25)
        prob_q75 = predicted.predicted_prob.quantile(.75)

    elif outcome=='reinspect':
        inspected = predicted[predicted['call_past_3m']==1]
        ptop = inspected.head(n=100)
        precision_at_n = precision_score(ptop['calls'], ptop['predicted'])
        recall_at_n = recall_score(ptop['calls'], ptop['predicted'])
        roc_auc = roc_auc_score(predicted['calls'], predicted['predicted_prob'])
        
        prob_avg = predicted.predicted_prob.mean()
        prob_med = predicted.predicted_prob.quantile(.5)
        prob_min = predicted.predicted_prob.min()
        prob_max = predicted.predicted_prob.max()
        prob_q25 = predicted.predicted_prob.quantile(.25)
        prob_q75 = predicted.predicted_prob.quantile(.75)
        
    elif outcome=='activity':
        inspected = predicted[predicted['calls']==1]
        ptop = inspected.head(n=100)
        precision_at_n = precision_score(ptop[outcome], ptop['predicted'])
        recall_at_n = recall_score(ptop[outcome], ptop['predicted'])
        roc_auc = roc_auc_score(inspected[outcome], inspected['predicted_prob'])

        prob_avg = predicted.predicted_prob.mean()
        prob_med = predicted.predicted_prob.quantile(.5)
        prob_min = predicted.predicted_prob.min()
        prob_max = predicted.predicted_prob.max()
        prob_q25 = predicted.predicted_prob.quantile(.25)
        prob_q75 = predicted.predicted_prob.quantile(.75)
        
    result = {'outcome': [outcome], 'year': [year], 'month': [month], 'clf': [model], 
              'precision_at_n': [precision_at_n], 'recall_at_n': [recall_at_n], 'auc': [roc_auc],
             'avg_pprob': [prob_avg], 'min_pprob': [prob_min], 'max_pprob': [prob_max],
             'q25_pprob': [prob_q25], 'med_pprob': [prob_med], 'q75_pprob': [prob_q75]
             }
    result = pd.DataFrame(result)
    
    for p in params:
        result[p] = params[p]

    results = results.append(result)

    if model=='LR':
        fimps = pd.DataFrame({'features': X_train.columns, 'coefficients': clf.coef_[0]}).sort_values('coefficients', ascending=False)
    else:
        fimps = pd.DataFrame({'features': X_train.columns, 'importances': clf.feature_importances_}).sort_values('importances', ascending=False)

    print ('Finished modeling '+str(month)+'-'+str(year)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))
    return results, predicted, fimps

## Train Models

In [8]:
## Import Census blocks
blocks = gpd.read_file('data/Census_Blocks__2010.geojson')

## DataFrame to hold model performance data for each set of hyperparameters
all_results = pd.DataFrame()
all_importances = pd.DataFrame()

## Looping through outcomes 
for outcome in outcomes:

    print ('Started '+outcome+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))
    
    ## Looping through different date-offsets/time-windows
    for offset in date_offset:
        
        print ('\nStarted models for Offset '+str(offset)+' month(s) at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))
        
        data = data_prep(date_offset=offset)
        
        ## Getting full set of years from data
        years = np.sort(np.arange(2016, pd.to_datetime('today').year + 1))
        years = years[~np.isnan(years)]
        
        ## Getting min and max years to define bounds of cross-validation
        min_year = np.min(years)
        max_year = np.max(years)
        min_month = np.min(data[data.year==min_year].month.astype(int).unique())
        max_month = np.max(data[data.year==max_year].month.astype(int).unique())
        min_date = pd.datetime(min_year, min_month, 1)
        max_date = pd.datetime(max_year, max_month, 1)
        print (min_date, max_date)
        
        
        ## Looping through model types
        for i in np.arange(0, len(models_to_run)):
            parameter_values = grid[models_to_run[i]]

            print ('\nStarted '+models_to_run[i]+' models at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

            ## Looping through sets of hyperparameters for the model
            for p, params in enumerate(ParameterGrid(parameter_values)):

                print ('\nStarted '+models_to_run[i]+' with param set '+str(p)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))


                ## DataFrames for model performance data and feature importances 
                results = pd.DataFrame()
                importances = pd.DataFrame()

                ## Assigning model typle and hyperparameters
                clf = clfs[models_to_run[i]]
                clf.set_params(**params)

                filename = outcome+'_'+models_to_run[i]+'_Params'+str(p)+'_Offset'+str(offset)
                
                ## Looping through years
                for year in years:
                    
                    months = np.sort(data[data.year==year].month.astype(int).unique())

                    ## Looping through months within year
                    for month in months:
                    
                        d = pd.datetime(year, month, 1)                        
                        
                        if d < pd.datetime(2016, 8, 1): 

                            print (d, 'too soon')
                            continue

                        elif d > pd.datetime(2017, 8, 1): 

                            print (d, 'too late')
                            continue

                        else:

                            print ('\nStarted models for '+str(year)+'-'+str(month)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))                            
                            print (d)

                            ## Get data from month-year
                            X_train, y_train, X_test, y_test = pull_data(data, outcome, year, month, date_offset=offset)
                            ## Run model and get performance data
                            result, predicted, fimps = train_model(outcome, year, month, clf, models_to_run[i], params, X_train, y_train, X_test, y_test)
                            ## Append performance data
                            results = results.append(result)
                            importances = importances.append(fimps)    
                            
                            ## Data visualizations
                            bins = 10
                            binwidth = 1.0/bins
                            cutpoints = np.arange(0.0*binwidth, 1 + binwidth, binwidth)  # cutpoints between score bins
                            labs = np.arange(0,1.0,binwidth)

                            inspected = predicted[predicted.calls==1]
                            n = inspected.groupby(pd.cut(x=inspected['predicted_prob'], bins=cutpoints))['activity'].count()
                            z = inspected.groupby(pd.cut(x=inspected['predicted_prob'], bins=cutpoints))['activity'].mean()
                            n.index = labs #added to deal with a change in pandas 0.16.1 that creates categorical indices after pd.cut()
                            z.index = labs #added to deal with a change in pandas 0.16.1 that creates categorical indices after pd.cut()

                            labs = ['(0,10]', '(10,20]', '(20,30]', '(30,40]', '(40,50]', '(50,60]', '(60,70]', '(70,80]', '(80,90]', '(90,100]']
                        
                            mname = dict()
                            mname[1] = 'January'
                            mname[2] = 'February'
                            mname[3] = 'March'
                            mname[4] = 'April'
                            mname[5] = 'May'
                            mname[6] = 'June'
                            mname[7] = 'July'
                            mname[8] = 'August'
                            mname[9] = 'September'
                            mname[10] = 'October'
                            mname[11] = 'November'
                            mname[12] = 'December'
                            
                            fig, ax = plt.subplots()
                            z[n > 1].plot(kind='line',rot=0,marker='.',markersize=2, color='red') 
                            plt.plot([0,1], [0.05, 1.05], '--', color='black')
                            ax.set_xlabel('Predicted Probability of Finding Rat Burrows')
                            ax.set_ylabel('Percentage of Inspections\n where Rat Burrows Found')
                            ax.set_title(mname[month]+' '+str(year))
                            ax.set_xticks(np.arange(0,1,.1))
                            ax.set_yticks(np.arange(.1,1.1,.1))
                            ax.set_xticklabels(labs)
                            ax.set_yticklabels(np.arange(10,110,10))
                            ax.set_xlim(0,.9)
                            ax.set_ylim(0,1)

                            plt.savefig('figures/decile_'+str(year)+'_'+str(month)+'.png', bbox_inches = 'tight')
                            plt.close(fig)
                            
                ## Get full set of dates
                dates = []
                for year in results.year.unique():
                    year = int(year)
                    this_year = results[results.year==year]
                    months = np.sort(this_year.month.astype(int).unique())
                    for month in months:
                        dates.append(str(month)+'-'+str(year))

                ## Distribution of Performance Metric            
                fig, ax = plt.subplots()
                sns.kdeplot(results['precision_at_n'], shade=True)
                ax.set_title('Distribution of Performance Metric')
                ax.set_xlabel('Precision at N')
                ax.set_yticklabels('')
                plt.savefig('figures/'+filename+'_perf_metric_distn.png', bbox_inches='tight')
                plt.close(fig)

                ## Distribution of performance metric across months            
                fig, ax = plt.subplots()
                sns.barplot(np.arange(0,len(results)), results['precision_at_n'], color='blue', alpha=0.5)
                ax.set_title('Distribution of Performance Metric Across Months')
                ax.set_ylim(0,1)
                ax.set_ylabel('Precision at N')
                ax.set_xticklabels(dates)
                plt.xticks(rotation=70)
                plt.savefig('figures/'+filename+'_perf_metric_distn_bymonth.png', bbox_inches='tight')
                plt.close(fig)
                    
                pd.DataFrame(predicted.describe()).to_csv('model_outputs/'+filename+'_descriptive_stats.csv')
                results.to_csv('model_outputs/'+filename+'_model_perf.csv', index=False)
                importances.to_csv('model_outputs/'+filename+'_importances.csv', index=False)
                predicted.to_csv('model_outputs/'+filename+'_predictions.csv')

                print ('Finished '+models_to_run[i]+' with param set '+str(p)+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

            print ('Finished all '+models_to_run[i]+' models at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

        print ('Finished models for Offset '+str(offset)+' month(s) at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

    print ('Finished '+outcome+' at '+time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime()))

Started activity at Wed, 11 Jul 2018 21:22:25 +0000

Started models for Offset 3 month(s) at Wed, 11 Jul 2018 21:22:25 +0000
Finished prepping data for 8-2015 at Wed, 11 Jul 2018 21:22:30 +0000
Finished prepping data for 9-2015 at Wed, 11 Jul 2018 21:22:31 +0000
Finished prepping data for 10-2015 at Wed, 11 Jul 2018 21:22:31 +0000
Finished prepping data for 11-2015 at Wed, 11 Jul 2018 21:22:31 +0000
Finished prepping data for 12-2015 at Wed, 11 Jul 2018 21:22:31 +0000
Finished prepping data for 1-2016 at Wed, 11 Jul 2018 21:22:31 +0000
Finished prepping data for 2-2016 at Wed, 11 Jul 2018 21:22:32 +0000
Finished prepping data for 3-2016 at Wed, 11 Jul 2018 21:22:32 +0000
Finished prepping data for 4-2016 at Wed, 11 Jul 2018 21:22:32 +0000
Finished prepping data for 5-2016 at Wed, 11 Jul 2018 21:22:32 +0000
Finished prepping data for 6-2016 at Wed, 11 Jul 2018 21:22:33 +0000
Finished prepping data for 7-2016 at Wed, 11 Jul 2018 21:22:33 +0000
Finished prepping data for 8-2016 at Wed, 11

Finished modeling 9-2016 at Wed, 11 Jul 2018 21:26:40 +0000

Started models for 2016-10 at Wed, 11 Jul 2018 21:26:42 +0000
2016-10-01 00:00:00
Finished modeling 10-2016 at Wed, 11 Jul 2018 21:26:43 +0000

Started models for 2016-11 at Wed, 11 Jul 2018 21:26:45 +0000
2016-11-01 00:00:00
Finished modeling 11-2016 at Wed, 11 Jul 2018 21:26:47 +0000

Started models for 2016-12 at Wed, 11 Jul 2018 21:26:48 +0000
2016-12-01 00:00:00
Finished modeling 12-2016 at Wed, 11 Jul 2018 21:26:51 +0000

Started models for 2017-1 at Wed, 11 Jul 2018 21:26:52 +0000
2017-01-01 00:00:00
Finished modeling 1-2017 at Wed, 11 Jul 2018 21:26:55 +0000

Started models for 2017-2 at Wed, 11 Jul 2018 21:26:56 +0000
2017-02-01 00:00:00
Finished modeling 2-2017 at Wed, 11 Jul 2018 21:26:59 +0000

Started models for 2017-3 at Wed, 11 Jul 2018 21:27:00 +0000
2017-03-01 00:00:00
Finished modeling 3-2017 at Wed, 11 Jul 2018 21:27:04 +0000

Started models for 2017-4 at Wed, 11 Jul 2018 21:27:05 +0000
2017-04-01 00:00:00


Finished modeling 2-2017 at Wed, 11 Jul 2018 21:30:22 +0000

Started models for 2017-3 at Wed, 11 Jul 2018 21:30:23 +0000
2017-03-01 00:00:00
Finished modeling 3-2017 at Wed, 11 Jul 2018 21:30:28 +0000

Started models for 2017-4 at Wed, 11 Jul 2018 21:30:29 +0000
2017-04-01 00:00:00
Finished modeling 4-2017 at Wed, 11 Jul 2018 21:30:34 +0000

Started models for 2017-5 at Wed, 11 Jul 2018 21:30:35 +0000
2017-05-01 00:00:00
Finished modeling 5-2017 at Wed, 11 Jul 2018 21:30:40 +0000

Started models for 2017-6 at Wed, 11 Jul 2018 21:30:42 +0000
2017-06-01 00:00:00
Finished modeling 6-2017 at Wed, 11 Jul 2018 21:30:47 +0000

Started models for 2017-7 at Wed, 11 Jul 2018 21:30:49 +0000
2017-07-01 00:00:00
Finished modeling 7-2017 at Wed, 11 Jul 2018 21:30:55 +0000

Started models for 2017-8 at Wed, 11 Jul 2018 21:30:56 +0000
2017-08-01 00:00:00
Finished modeling 8-2017 at Wed, 11 Jul 2018 21:31:03 +0000
2017-09-01 00:00:00 too late
2017-10-01 00:00:00 too late
2017-11-01 00:00:00 too late
2

Finished modeling 7-2017 at Wed, 11 Jul 2018 21:34:47 +0000

Started models for 2017-8 at Wed, 11 Jul 2018 21:34:48 +0000
2017-08-01 00:00:00
Finished modeling 8-2017 at Wed, 11 Jul 2018 21:34:54 +0000
2017-09-01 00:00:00 too late
2017-10-01 00:00:00 too late
2017-11-01 00:00:00 too late
2017-12-01 00:00:00 too late
2018-01-01 00:00:00 too late
2018-02-01 00:00:00 too late
2018-03-01 00:00:00 too late
2018-04-01 00:00:00 too late
2018-05-01 00:00:00 too late
2018-06-01 00:00:00 too late
2018-07-01 00:00:00 too late
Finished RF with param set 8 at Wed, 11 Jul 2018 21:34:58 +0000

Started RF with param set 9 at Wed, 11 Jul 2018 21:34:58 +0000
2016-01-01 00:00:00 too soon
2016-02-01 00:00:00 too soon
2016-03-01 00:00:00 too soon
2016-04-01 00:00:00 too soon
2016-05-01 00:00:00 too soon
2016-06-01 00:00:00 too soon
2016-07-01 00:00:00 too soon

Started models for 2016-8 at Wed, 11 Jul 2018 21:34:58 +0000
2016-08-01 00:00:00
Finished modeling 8-2016 at Wed, 11 Jul 2018 21:35:01 +0000

Start